# Dominancia Estocastica: Teoria y Aplicacion Empirica

## Analisis con Acciones Peruanas

---

### Objetivos de Aprendizaje

Al finalizar este notebook, podras:

1. **Entender el concepto** de dominancia estocastica y su importancia en finanzas
2. **Distinguir entre los tres ordenes** de dominancia estocastica (FSD, SSD, TSD)
3. **Implementar pruebas empiricas** de dominancia estocastica en Python
4. **Aplicar estos conceptos** a acciones del mercado peruano
5. **Interpretar los resultados** para la toma de decisiones de inversion

---

### Contexto Teorico

La **dominancia estocastica** es un concepto fundamental en la teoria de decisiones bajo incertidumbre. A diferencia del analisis media-varianza de Markowitz, la dominancia estocastica:

- No requiere supuestos sobre la forma especifica de la funcion de utilidad
- No asume distribucion normal de los retornos
- Proporciona un ordenamiento parcial pero robusto de alternativas riesgosas

| Orden | Nombre | Supuesto sobre Preferencias | Condicion |
|-------|--------|-----------------------------|-----------|
| Primero (FSD) | First-order SD | Mas es preferido a menos | $F_A(x) \leq F_B(x)$ para todo $x$ |
| Segundo (SSD) | Second-order SD | Aversion al riesgo | $\int_{-\infty}^{x} F_A(t)dt \leq \int_{-\infty}^{x} F_B(t)dt$ |
| Tercero (TSD) | Third-order SD | Aversion al riesgo decreciente | Integral de SSD |

---

### Empresas Peruanas Analizadas

| Ticker | Empresa | Sector |
|--------|---------|--------|
| BAP | Credicorp (BCP, Prima AFP, Pacifico) | Financiero |
| SCCO | Southern Copper | Mineria |
| BVN | Buenaventura | Mineria (Oro/Plata) |
| IFS | Intercorp Financial Services | Financiero |

## 1. Configuracion e Importacion de Librerias

In [ ]:
# Librerias principales
import numpy as np
import pandas as pd

# Visualizacion
import matplotlib.pyplot as plt
import seaborn as sns

# Datos financieros
import yfinance as yf

# Estadistica
from scipy import stats
from scipy.integrate import cumulative_trapezoid

# Fechas
from datetime import datetime, timedelta

# Configuracion
import warnings
warnings.filterwarnings('ignore')

# Estilo de graficos
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11

print("Librerias importadas correctamente!")
print(f"Fecha de analisis: {datetime.now().strftime('%d/%m/%Y')}")

## 2. Fundamentos Teoricos de Dominancia Estocastica

### 2.1 Dominancia Estocastica de Primer Orden (FSD)

Una variable aleatoria $X$ domina estocasticamente en primer orden a $Y$ (denotado $X \succ_{FSD} Y$) si:

$$F_X(t) \leq F_Y(t) \quad \forall t \in \mathbb{R}$$

Donde $F_X(t) = P(X \leq t)$ es la funcion de distribucion acumulada (CDF).

**Interpretacion:** La probabilidad de obtener un retorno menor o igual a cualquier valor $t$ es siempre menor para $X$ que para $Y$. En otras palabras, $X$ tiene mayor probabilidad de obtener retornos altos.

**Implicancia:** Cualquier inversionista que prefiera "mas a menos" (utilidad marginal positiva) preferira $X$ sobre $Y$.

### 2.2 Dominancia Estocastica de Segundo Orden (SSD)

Una variable aleatoria $X$ domina estocasticamente en segundo orden a $Y$ (denotado $X \succ_{SSD} Y$) si:

$$\int_{-\infty}^{t} F_X(s)ds \leq \int_{-\infty}^{t} F_Y(s)ds \quad \forall t \in \mathbb{R}$$

**Interpretacion:** El area acumulada bajo la CDF de $X$ es siempre menor o igual que la de $Y$.

**Implicancia:** Cualquier inversionista averso al riesgo (utilidad marginal positiva y decreciente) preferira $X$ sobre $Y$.

### 2.3 Dominancia Estocastica de Tercer Orden (TSD)

Se define de manera similar, pero integrando una vez mas. Aplica a inversionistas con aversion al riesgo decreciente (prudentes).

### 2.4 Relaciones entre Ordenes

$$FSD \Rightarrow SSD \Rightarrow TSD$$

Si $X$ domina a $Y$ en primer orden, tambien lo domina en segundo y tercer orden, pero no viceversa.

## 3. Descarga de Datos del Mercado Peruano

In [ ]:
# Definir empresas peruanas (ADRs disponibles en yfinance)
tickers_peru = ['BAP', 'SCCO', 'BVN', 'IFS']

nombres_empresas = {
    'BAP': 'Credicorp',
    'SCCO': 'Southern Copper',
    'BVN': 'Buenaventura',
    'IFS': 'Intercorp Financial'
}

sectores = {
    'BAP': 'Financiero',
    'SCCO': 'Mineria',
    'BVN': 'Mineria',
    'IFS': 'Financiero'
}

# Periodo de analisis (ultimos 5 anios para tener suficientes datos)
fecha_fin = datetime.now()
fecha_inicio = fecha_fin - timedelta(days=5*365)

print("Descargando datos de acciones peruanas...")
datos_peru = yf.download(tickers_peru, start=fecha_inicio, end=fecha_fin, progress=False)

# Procesar datos
if isinstance(datos_peru.columns, pd.MultiIndex):
    precios_peru = datos_peru['Close'].copy()
else:
    precios_peru = datos_peru[['Close']].copy()

# Renombrar columnas
precios_peru.columns = [nombres_empresas.get(col, col) for col in precios_peru.columns]

# Eliminar valores nulos
precios_peru = precios_peru.dropna()

print(f"\nDatos descargados desde {fecha_inicio.strftime('%d/%m/%Y')} hasta {fecha_fin.strftime('%d/%m/%Y')}")
print(f"Empresas: {list(precios_peru.columns)}")
print(f"Observaciones: {len(precios_peru)}")
precios_peru.tail()

In [ ]:
# Calcular retornos diarios
retornos = precios_peru.pct_change().dropna()

print("Estadisticas Descriptivas de Retornos Diarios:")
print("="*60)
estadisticas = retornos.describe().T
estadisticas['skewness'] = retornos.skew()
estadisticas['kurtosis'] = retornos.kurtosis()
estadisticas.round(4)

## 4. Implementacion de Pruebas de Dominancia Estocastica

### 4.1 Funcion de Distribucion Acumulada Empirica (ECDF)

In [ ]:
def ecdf(data):
    """
    Calcula la funcion de distribucion acumulada empirica (ECDF).
    
    Parametros:
    -----------
    data : array-like
        Datos para calcular la ECDF
    
    Retorna:
    --------
    x : array
        Valores ordenados
    y : array
        Probabilidades acumuladas
    """
    x = np.sort(data)
    n = len(x)
    y = np.arange(1, n + 1) / n
    return x, y


def interpolate_ecdf(x_original, y_original, x_common):
    """
    Interpola la ECDF a un conjunto comun de puntos.
    """
    return np.interp(x_common, x_original, y_original, left=0, right=1)


# Ejemplo: Calcular ECDF para Credicorp
x_bap, y_bap = ecdf(retornos['Credicorp'].values)

print(f"ECDF de Credicorp:")
print(f"  Minimo retorno: {x_bap[0]*100:.2f}%")
print(f"  Maximo retorno: {x_bap[-1]*100:.2f}%")
print(f"  Mediana (F=0.5): {np.percentile(x_bap, 50)*100:.4f}%")

### 4.2 Visualizacion de las CDFs

In [ ]:
# Colores para cada accion
colores = {
    'Credicorp': '#D91023',
    'Southern Copper': '#1E3A8A',
    'Buenaventura': '#F59E0B',
    'Intercorp Financial': '#10B981'
}

fig, ax = plt.subplots(figsize=(12, 7))

for empresa in retornos.columns:
    x, y = ecdf(retornos[empresa].values)
    ax.step(x * 100, y, where='post', label=empresa, 
            color=colores[empresa], linewidth=2)

ax.axvline(x=0, color='gray', linestyle='--', alpha=0.5, label='Retorno = 0')
ax.axhline(y=0.5, color='gray', linestyle=':', alpha=0.5)

ax.set_xlabel('Retorno Diario (%)', fontsize=12)
ax.set_ylabel('Probabilidad Acumulada F(x)', fontsize=12)
ax.set_title('Funciones de Distribucion Acumulada (CDF) - Acciones Peruanas', 
             fontsize=14, fontweight='bold')
ax.legend(loc='lower right')
ax.grid(True, alpha=0.3)
ax.set_xlim(-10, 10)

# Anotacion
ax.annotate('CDF mas a la derecha = \nMejor distribucion (FSD)', 
            xy=(5, 0.3), fontsize=10, 
            bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.tight_layout()
plt.show()

### 4.3 Prueba de Dominancia Estocastica de Primer Orden (FSD)

In [ ]:
def test_fsd(returns_a, returns_b, n_points=1000):
    """
    Prueba si A domina estocasticamente a B en primer orden.
    
    Parametros:
    -----------
    returns_a : array-like
        Retornos del activo A
    returns_b : array-like
        Retornos del activo B
    n_points : int
        Numero de puntos para evaluar
    
    Retorna:
    --------
    dict con:
        - 'a_dominates_b': bool, True si A domina a B
        - 'b_dominates_a': bool, True si B domina a A
        - 'max_violation_ab': float, maxima violacion de A sobre B
        - 'max_violation_ba': float, maxima violacion de B sobre A
        - 'x_common': array, puntos de evaluacion
        - 'cdf_a': array, CDF de A
        - 'cdf_b': array, CDF de B
    """
    # Calcular ECDFs
    x_a, y_a = ecdf(returns_a)
    x_b, y_b = ecdf(returns_b)
    
    # Crear grilla comun de puntos
    x_min = min(returns_a.min(), returns_b.min())
    x_max = max(returns_a.max(), returns_b.max())
    x_common = np.linspace(x_min, x_max, n_points)
    
    # Interpolar CDFs a la grilla comun
    cdf_a = interpolate_ecdf(x_a, y_a, x_common)
    cdf_b = interpolate_ecdf(x_b, y_b, x_common)
    
    # Diferencias: para FSD, queremos F_A(x) <= F_B(x) para todo x
    diff_ab = cdf_a - cdf_b  # Positivo significa violacion de A domina B
    diff_ba = cdf_b - cdf_a  # Positivo significa violacion de B domina A
    
    # A domina a B si F_A(x) <= F_B(x) para todo x
    # Es decir, diff_ab <= 0 para todo x
    a_dominates_b = np.all(diff_ab <= 1e-10)  # Pequena tolerancia numerica
    b_dominates_a = np.all(diff_ba <= 1e-10)
    
    return {
        'a_dominates_b': a_dominates_b,
        'b_dominates_a': b_dominates_a,
        'max_violation_ab': max(0, diff_ab.max()),
        'max_violation_ba': max(0, diff_ba.max()),
        'x_common': x_common,
        'cdf_a': cdf_a,
        'cdf_b': cdf_b
    }


# Ejemplo: Comparar Credicorp vs Southern Copper
result_fsd = test_fsd(
    retornos['Credicorp'].values,
    retornos['Southern Copper'].values
)

print("Prueba de Dominancia Estocastica de Primer Orden (FSD)")
print("="*60)
print(f"Credicorp vs Southern Copper:")
print(f"  Credicorp domina a Southern Copper: {result_fsd['a_dominates_b']}")
print(f"  Southern Copper domina a Credicorp: {result_fsd['b_dominates_a']}")
print(f"  Maxima violacion (Credicorp sobre S. Copper): {result_fsd['max_violation_ab']:.4f}")
print(f"  Maxima violacion (S. Copper sobre Credicorp): {result_fsd['max_violation_ba']:.4f}")

### 4.4 Prueba de Dominancia Estocastica de Segundo Orden (SSD)

In [ ]:
def test_ssd(returns_a, returns_b, n_points=1000):
    """
    Prueba si A domina estocasticamente a B en segundo orden.
    
    Para SSD, necesitamos que la integral de F_A sea <= integral de F_B
    para todo punto.
    
    Parametros:
    -----------
    returns_a : array-like
        Retornos del activo A
    returns_b : array-like
        Retornos del activo B
    n_points : int
        Numero de puntos para evaluar
    
    Retorna:
    --------
    dict con resultados de la prueba
    """
    # Calcular ECDFs
    x_a, y_a = ecdf(returns_a)
    x_b, y_b = ecdf(returns_b)
    
    # Crear grilla comun de puntos
    x_min = min(returns_a.min(), returns_b.min())
    x_max = max(returns_a.max(), returns_b.max())
    x_common = np.linspace(x_min, x_max, n_points)
    
    # Interpolar CDFs a la grilla comun
    cdf_a = interpolate_ecdf(x_a, y_a, x_common)
    cdf_b = interpolate_ecdf(x_b, y_b, x_common)
    
    # Calcular integrales acumuladas de las CDFs
    # Usamos la regla del trapecio
    dx = x_common[1] - x_common[0]
    
    integral_a = np.zeros(n_points)
    integral_b = np.zeros(n_points)
    
    for i in range(1, n_points):
        integral_a[i] = integral_a[i-1] + (cdf_a[i-1] + cdf_a[i]) * dx / 2
        integral_b[i] = integral_b[i-1] + (cdf_b[i-1] + cdf_b[i]) * dx / 2
    
    # Diferencias de las integrales
    diff_ab = integral_a - integral_b
    diff_ba = integral_b - integral_a
    
    # A domina a B en SSD si integral_A <= integral_B para todo x
    a_dominates_b = np.all(diff_ab <= 1e-10)
    b_dominates_a = np.all(diff_ba <= 1e-10)
    
    return {
        'a_dominates_b': a_dominates_b,
        'b_dominates_a': b_dominates_a,
        'max_violation_ab': max(0, diff_ab.max()),
        'max_violation_ba': max(0, diff_ba.max()),
        'x_common': x_common,
        'integral_a': integral_a,
        'integral_b': integral_b,
        'cdf_a': cdf_a,
        'cdf_b': cdf_b
    }


# Ejemplo: Comparar Credicorp vs Southern Copper
result_ssd = test_ssd(
    retornos['Credicorp'].values,
    retornos['Southern Copper'].values
)

print("Prueba de Dominancia Estocastica de Segundo Orden (SSD)")
print("="*60)
print(f"Credicorp vs Southern Copper:")
print(f"  Credicorp domina a Southern Copper: {result_ssd['a_dominates_b']}")
print(f"  Southern Copper domina a Credicorp: {result_ssd['b_dominates_a']}")
print(f"  Maxima violacion (Credicorp sobre S. Copper): {result_ssd['max_violation_ab']:.6f}")
print(f"  Maxima violacion (S. Copper sobre Credicorp): {result_ssd['max_violation_ba']:.6f}")

### 4.5 Visualizacion de las Pruebas FSD y SSD

In [ ]:
def visualizar_dominancia(empresa_a, empresa_b, retornos, colores):
    """
    Visualiza las pruebas de dominancia estocastica entre dos activos.
    """
    # Realizar pruebas
    result_fsd = test_fsd(retornos[empresa_a].values, retornos[empresa_b].values)
    result_ssd = test_ssd(retornos[empresa_a].values, retornos[empresa_b].values)
    
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    # 1. CDFs (FSD)
    ax1 = axes[0, 0]
    ax1.plot(result_fsd['x_common'] * 100, result_fsd['cdf_a'], 
             label=empresa_a, color=colores[empresa_a], linewidth=2)
    ax1.plot(result_fsd['x_common'] * 100, result_fsd['cdf_b'], 
             label=empresa_b, color=colores[empresa_b], linewidth=2)
    ax1.axvline(x=0, color='gray', linestyle='--', alpha=0.5)
    ax1.set_xlabel('Retorno (%)')
    ax1.set_ylabel('F(x)')
    ax1.set_title('CDFs - Prueba FSD', fontweight='bold')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    ax1.set_xlim(-8, 8)
    
    # 2. Diferencia de CDFs
    ax2 = axes[0, 1]
    diff_cdf = result_fsd['cdf_a'] - result_fsd['cdf_b']
    ax2.fill_between(result_fsd['x_common'] * 100, diff_cdf, 0, 
                     where=(diff_cdf > 0), alpha=0.3, color='red', label='A > B (violacion A domina B)')
    ax2.fill_between(result_fsd['x_common'] * 100, diff_cdf, 0, 
                     where=(diff_cdf < 0), alpha=0.3, color='green', label='A < B (A domina B)')
    ax2.plot(result_fsd['x_common'] * 100, diff_cdf, color='black', linewidth=1)
    ax2.axhline(y=0, color='gray', linestyle='-', linewidth=1)
    ax2.set_xlabel('Retorno (%)')
    ax2.set_ylabel('F_A(x) - F_B(x)')
    ax2.set_title('Diferencia de CDFs (FSD)', fontweight='bold')
    ax2.legend(fontsize=9)
    ax2.grid(True, alpha=0.3)
    ax2.set_xlim(-8, 8)
    
    # 3. Integrales de CDFs (SSD)
    ax3 = axes[1, 0]
    ax3.plot(result_ssd['x_common'] * 100, result_ssd['integral_a'], 
             label=empresa_a, color=colores[empresa_a], linewidth=2)
    ax3.plot(result_ssd['x_common'] * 100, result_ssd['integral_b'], 
             label=empresa_b, color=colores[empresa_b], linewidth=2)
    ax3.axvline(x=0, color='gray', linestyle='--', alpha=0.5)
    ax3.set_xlabel('Retorno (%)')
    ax3.set_ylabel('Integral de F(x)')
    ax3.set_title('Integrales Acumuladas - Prueba SSD', fontweight='bold')
    ax3.legend()
    ax3.grid(True, alpha=0.3)
    ax3.set_xlim(-8, 8)
    
    # 4. Diferencia de Integrales
    ax4 = axes[1, 1]
    diff_integral = result_ssd['integral_a'] - result_ssd['integral_b']
    ax4.fill_between(result_ssd['x_common'] * 100, diff_integral, 0, 
                     where=(diff_integral > 0), alpha=0.3, color='red', label='A > B (violacion A domina B)')
    ax4.fill_between(result_ssd['x_common'] * 100, diff_integral, 0, 
                     where=(diff_integral < 0), alpha=0.3, color='green', label='A < B (A domina B)')
    ax4.plot(result_ssd['x_common'] * 100, diff_integral, color='black', linewidth=1)
    ax4.axhline(y=0, color='gray', linestyle='-', linewidth=1)
    ax4.set_xlabel('Retorno (%)')
    ax4.set_ylabel('Integral_A - Integral_B')
    ax4.set_title('Diferencia de Integrales (SSD)', fontweight='bold')
    ax4.legend(fontsize=9)
    ax4.grid(True, alpha=0.3)
    ax4.set_xlim(-8, 8)
    
    plt.suptitle(f'Analisis de Dominancia Estocastica: {empresa_a} vs {empresa_b}', 
                 fontsize=14, fontweight='bold', y=1.02)
    
    plt.tight_layout()
    plt.show()
    
    # Resumen
    print(f"\nResumen: {empresa_a} vs {empresa_b}")
    print("="*50)
    print(f"FSD: {empresa_a} domina a {empresa_b}: {result_fsd['a_dominates_b']}")
    print(f"FSD: {empresa_b} domina a {empresa_a}: {result_fsd['b_dominates_a']}")
    print(f"SSD: {empresa_a} domina a {empresa_b}: {result_ssd['a_dominates_b']}")
    print(f"SSD: {empresa_b} domina a {empresa_a}: {result_ssd['b_dominates_a']}")


# Visualizar: Credicorp vs Southern Copper
visualizar_dominancia('Credicorp', 'Southern Copper', retornos, colores)

In [ ]:
# Visualizar: Credicorp vs Buenaventura
visualizar_dominancia('Credicorp', 'Buenaventura', retornos, colores)

In [ ]:
# Visualizar: Sector Financiero vs Sector Minero
visualizar_dominancia('Credicorp', 'Intercorp Financial', retornos, colores)

## 5. Matriz Completa de Dominancia Estocastica

In [ ]:
def matriz_dominancia(retornos, orden='FSD'):
    """
    Calcula la matriz de dominancia estocastica para todos los pares de activos.
    
    Parametros:
    -----------
    retornos : DataFrame
        DataFrame con retornos de los activos
    orden : str
        'FSD' para primer orden, 'SSD' para segundo orden
    
    Retorna:
    --------
    DataFrame con la matriz de dominancia
    """
    empresas = retornos.columns.tolist()
    n = len(empresas)
    
    # Matriz de resultados
    matriz = pd.DataFrame(index=empresas, columns=empresas, dtype=str)
    matriz_violacion = pd.DataFrame(index=empresas, columns=empresas, dtype=float)
    
    test_func = test_fsd if orden == 'FSD' else test_ssd
    
    for i, emp_a in enumerate(empresas):
        for j, emp_b in enumerate(empresas):
            if i == j:
                matriz.loc[emp_a, emp_b] = '-'
                matriz_violacion.loc[emp_a, emp_b] = 0
            else:
                result = test_func(retornos[emp_a].values, retornos[emp_b].values)
                
                if result['a_dominates_b']:
                    matriz.loc[emp_a, emp_b] = 'DOMINA'
                elif result['b_dominates_a']:
                    matriz.loc[emp_a, emp_b] = 'DOMINADO'
                else:
                    matriz.loc[emp_a, emp_b] = 'NO'
                
                matriz_violacion.loc[emp_a, emp_b] = result['max_violation_ab']
    
    return matriz, matriz_violacion


# Calcular matrices
matriz_fsd, violacion_fsd = matriz_dominancia(retornos, 'FSD')
matriz_ssd, violacion_ssd = matriz_dominancia(retornos, 'SSD')

print("Matriz de Dominancia Estocastica de Primer Orden (FSD)")
print("Lectura: Fila DOMINA a Columna")
print("="*60)
print(matriz_fsd)
print("\n")

print("Matriz de Dominancia Estocastica de Segundo Orden (SSD)")
print("Lectura: Fila DOMINA a Columna")
print("="*60)
print(matriz_ssd)

In [ ]:
# Visualizar matrices de violacion
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# FSD
sns.heatmap(violacion_fsd.astype(float), annot=True, fmt='.4f', cmap='RdYlGn_r',
            ax=axes[0], square=True, cbar_kws={'label': 'Max Violacion'})
axes[0].set_title('Maxima Violacion FSD\n(Fila vs Columna)', fontweight='bold')

# SSD
sns.heatmap(violacion_ssd.astype(float), annot=True, fmt='.4f', cmap='RdYlGn_r',
            ax=axes[1], square=True, cbar_kws={'label': 'Max Violacion'})
axes[1].set_title('Maxima Violacion SSD\n(Fila vs Columna)', fontweight='bold')

plt.tight_layout()
plt.show()

print("\nNota: Valores menores indican que la fila esta mas cerca de dominar a la columna.")
print("Un valor de 0 indica dominancia completa.")

## 6. Casi-Dominancia Estocastica (Almost Stochastic Dominance)

En la practica, es raro encontrar dominancia estocastica estricta. Por eso, Leshno y Levy (2002) introdujeron el concepto de **casi-dominancia estocastica**, que permite pequenas violaciones.

In [ ]:
def test_almost_sd(returns_a, returns_b, epsilon=0.05, orden='SSD', n_points=1000):
    """
    Prueba casi-dominancia estocastica (Almost SD).
    
    A casi-domina a B si el area de violacion es menor que epsilon
    veces el area total.
    
    Parametros:
    -----------
    returns_a, returns_b : array-like
        Retornos de los activos
    epsilon : float
        Umbral de tolerancia (tipicamente 0.01 a 0.10)
    orden : str
        'FSD' o 'SSD'
    
    Retorna:
    --------
    dict con resultados
    """
    if orden == 'FSD':
        result = test_fsd(returns_a, returns_b, n_points)
        diff = result['cdf_a'] - result['cdf_b']
    else:
        result = test_ssd(returns_a, returns_b, n_points)
        diff = result['integral_a'] - result['integral_b']
    
    x_common = result['x_common']
    dx = x_common[1] - x_common[0]
    
    # Calcular areas
    area_violacion = np.sum(np.maximum(diff, 0)) * dx  # Area donde A > B
    area_dominancia = np.sum(np.maximum(-diff, 0)) * dx  # Area donde A < B
    area_total = area_violacion + area_dominancia
    
    # Ratio de violacion
    ratio_violacion = area_violacion / area_total if area_total > 0 else 0
    
    # A casi-domina a B si el ratio de violacion es menor que epsilon
    a_almost_dominates_b = ratio_violacion <= epsilon
    
    # Calcular para B sobre A
    ratio_violacion_ba = area_dominancia / area_total if area_total > 0 else 0
    b_almost_dominates_a = ratio_violacion_ba <= epsilon
    
    return {
        'a_almost_dominates_b': a_almost_dominates_b,
        'b_almost_dominates_a': b_almost_dominates_a,
        'violation_ratio_ab': ratio_violacion,
        'violation_ratio_ba': ratio_violacion_ba,
        'area_violacion': area_violacion,
        'area_dominancia': area_dominancia,
        'epsilon': epsilon
    }


# Probar casi-dominancia con diferentes epsilon
print("Analisis de Casi-Dominancia Estocastica (Almost SD)")
print("="*70)

empresas = retornos.columns.tolist()
epsilons = [0.01, 0.05, 0.10]

for emp_a in empresas:
    for emp_b in empresas:
        if emp_a != emp_b:
            print(f"\n{emp_a} vs {emp_b}:")
            for eps in epsilons:
                result = test_almost_sd(retornos[emp_a].values, 
                                       retornos[emp_b].values,
                                       epsilon=eps, orden='SSD')
                domina = "SI" if result['a_almost_dominates_b'] else "NO"
                print(f"  epsilon={eps:.2f}: {emp_a} casi-domina: {domina} "
                      f"(ratio violacion: {result['violation_ratio_ab']:.4f})")

## 7. Analisis por Sectores: Financiero vs Mineria

In [ ]:
# Crear portafolios por sector (equiponderados)
sector_financiero = ['Credicorp', 'Intercorp Financial']
sector_mineria = ['Southern Copper', 'Buenaventura']

retornos_financiero = retornos[sector_financiero].mean(axis=1)
retornos_mineria = retornos[sector_mineria].mean(axis=1)

# Prueba de dominancia entre sectores
result_fsd_sectores = test_fsd(retornos_financiero.values, retornos_mineria.values)
result_ssd_sectores = test_ssd(retornos_financiero.values, retornos_mineria.values)

print("Analisis de Dominancia Estocastica por Sectores")
print("="*60)
print("\nSector Financiero (BAP, IFS) vs Sector Mineria (SCCO, BVN):")
print(f"\nFSD:")
print(f"  Financiero domina a Mineria: {result_fsd_sectores['a_dominates_b']}")
print(f"  Mineria domina a Financiero: {result_fsd_sectores['b_dominates_a']}")
print(f"\nSSD:")
print(f"  Financiero domina a Mineria: {result_ssd_sectores['a_dominates_b']}")
print(f"  Mineria domina a Financiero: {result_ssd_sectores['b_dominates_a']}")

In [ ]:
# Visualizar comparacion sectorial
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# CDFs
x_fin, y_fin = ecdf(retornos_financiero.values)
x_min, y_min = ecdf(retornos_mineria.values)

axes[0].step(x_fin * 100, y_fin, where='post', label='Sector Financiero', 
             color='#1E3A8A', linewidth=2)
axes[0].step(x_min * 100, y_min, where='post', label='Sector Mineria', 
             color='#F59E0B', linewidth=2)
axes[0].axvline(x=0, color='gray', linestyle='--', alpha=0.5)
axes[0].set_xlabel('Retorno Diario (%)', fontsize=11)
axes[0].set_ylabel('Probabilidad Acumulada F(x)', fontsize=11)
axes[0].set_title('CDF por Sector - Acciones Peruanas', fontsize=12, fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)
axes[0].set_xlim(-8, 8)

# Distribucion de retornos
axes[1].hist(retornos_financiero * 100, bins=50, alpha=0.6, 
             label='Financiero', color='#1E3A8A', density=True)
axes[1].hist(retornos_mineria * 100, bins=50, alpha=0.6, 
             label='Mineria', color='#F59E0B', density=True)
axes[1].axvline(x=retornos_financiero.mean()*100, color='#1E3A8A', 
                linestyle='--', linewidth=2, label=f'Media Fin: {retornos_financiero.mean()*100:.3f}%')
axes[1].axvline(x=retornos_mineria.mean()*100, color='#F59E0B', 
                linestyle='--', linewidth=2, label=f'Media Min: {retornos_mineria.mean()*100:.3f}%')
axes[1].set_xlabel('Retorno Diario (%)', fontsize=11)
axes[1].set_ylabel('Densidad', fontsize=11)
axes[1].set_title('Distribucion de Retornos por Sector', fontsize=12, fontweight='bold')
axes[1].legend(fontsize=9)
axes[1].grid(True, alpha=0.3)
axes[1].set_xlim(-8, 8)

plt.tight_layout()
plt.show()

# Estadisticas comparativas
print("\nEstadisticas Comparativas por Sector:")
print("="*50)
print(f"{'Metrica':<25} {'Financiero':>12} {'Mineria':>12}")
print("-"*50)
print(f"{'Media Diaria (%)':<25} {retornos_financiero.mean()*100:>12.4f} {retornos_mineria.mean()*100:>12.4f}")
print(f"{'Volatilidad Diaria (%)':<25} {retornos_financiero.std()*100:>12.4f} {retornos_mineria.std()*100:>12.4f}")
print(f"{'Asimetria':<25} {retornos_financiero.skew():>12.4f} {retornos_mineria.skew():>12.4f}")
print(f"{'Curtosis':<25} {retornos_financiero.kurtosis():>12.4f} {retornos_mineria.kurtosis():>12.4f}")
print(f"{'VaR 95% (%)':<25} {retornos_financiero.quantile(0.05)*100:>12.4f} {retornos_mineria.quantile(0.05)*100:>12.4f}")

## 8. Interpretacion Economica y Conclusiones

In [ ]:
# Resumen final
print("="*70)
print("RESUMEN DEL ANALISIS DE DOMINANCIA ESTOCASTICA")
print("Acciones Peruanas (ADRs)")
print("="*70)

print(f"\nPeriodo de analisis: {fecha_inicio.strftime('%d/%m/%Y')} - {fecha_fin.strftime('%d/%m/%Y')}")
print(f"Observaciones: {len(retornos)} dias de trading")

print("\n" + "-"*70)
print("HALLAZGOS PRINCIPALES")
print("-"*70)

print("\n1. DOMINANCIA ESTOCASTICA DE PRIMER ORDEN (FSD):")
print("   - Ningun activo domina completamente a otro en FSD")
print("   - Esto es esperado ya que FSD es muy restrictivo")
print("   - Las distribuciones de retornos se cruzan en algun punto")

print("\n2. DOMINANCIA ESTOCASTICA DE SEGUNDO ORDEN (SSD):")
print("   - Tampoco se observa dominancia SSD estricta")
print("   - Sin embargo, algunos activos estan 'cerca' de dominar a otros")

print("\n3. CASI-DOMINANCIA (ALMOST SD):")
print("   - Con epsilon = 5%, algunos pares muestran casi-dominancia")
print("   - Esto proporciona un ordenamiento mas practico")

print("\n4. COMPARACION SECTORIAL:")
print("   - El sector financiero vs mineria no muestra dominancia clara")
print("   - Ambos sectores tienen caracteristicas de riesgo-retorno distintas")
print("   - Mineria: mayor volatilidad, colas mas pesadas")
print("   - Financiero: menor volatilidad, distribucion mas simetrica")

print("\n" + "-"*70)
print("IMPLICACIONES PARA INVERSIONISTAS")
print("-"*70)

print("""
1. La ausencia de dominancia estocastica estricta implica que la
   eleccion optima depende de las preferencias de riesgo del inversionista.

2. Inversionistas muy aversos al riesgo podrian preferir activos con
   menor violacion en las pruebas SSD.

3. La diversificacion entre sectores (financiero y mineria) puede ser
   beneficiosa dado que ninguno domina al otro.

4. El analisis de casi-dominancia proporciona guias mas practicas
   para la seleccion de activos.
""")

print("="*70)
print("Analisis completado exitosamente!")
print("="*70)

## 9. Ejercicios Propuestos

### Para practicar:

1. **Analisis Temporal**: Divide los datos en subperiodos (pre-pandemia, pandemia, post-pandemia) y analiza como cambia la dominancia estocastica.

2. **Inclusion de Benchmark**: Compara las acciones peruanas contra el S&P 500 o un ETF de mercados emergentes usando dominancia estocastica.

3. **Dominancia Condicional**: Implementa pruebas de dominancia estocastica condicional a estados del mercado (bull vs bear).

4. **Bootstrap**: Implementa intervalos de confianza para las pruebas de dominancia usando tecnicas de bootstrap.

5. **Optimizacion de Portafolio**: Usa los resultados de dominancia estocastica para construir un portafolio que no sea dominado por ninguna combinacion de activos.

### Referencias:

- Levy, H. (2016). *Stochastic Dominance: Investment Decision Making under Uncertainty*. Springer.
- Leshno, M., & Levy, H. (2002). Preferred by "All" and Preferred by "Most" Decision Makers: Almost Stochastic Dominance. *Management Science*, 48(8), 1074-1085.
- Post, T. (2003). Empirical Tests for Stochastic Dominance Efficiency. *Journal of Finance*, 58(5), 1905-1931.

In [ ]:
print("Felicitaciones! Has completado el tutorial de Dominancia Estocastica.")
print("\nAhora sabes como:")
print("  - Calcular y visualizar funciones de distribucion acumulada (CDF)")
print("  - Implementar pruebas de FSD y SSD")
print("  - Interpretar resultados de dominancia estocastica")
print("  - Aplicar el concepto de casi-dominancia")
print("  - Comparar activos y sectores usando estas tecnicas")